# Week 2 - Business and Data Understanding

# Activity

**Goal:** Replicate live demo notebook into individual lab notebook

**Data To Be Used:** week2_data_for_student_lab.csv.gz

In [1]:
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local[*]') \
        .appName('Basics') \
        .getOrCreate()

print(spark.version)

3.5.4


In [3]:
df_2018 = spark.read.csv(f"/content/drive/MyDrive/ TERM 7/MACHINE/churn_prediction/data/week2_data_for_student_lab.csv.gz", inferSchema=True, header=True)
df_2018.show(5)

+----------------+-----------+--------------------+------+-------+--------+------+-----+---------+-----------+------+-------------+
|transaction_date|member_type|           member_id|gender|product|quantity|amount|  age|card_type|branch_name|region|category_name|
+----------------+-----------+--------------------+------+-------+--------+------+-----+---------+-----------+------+-------------+
|      2018-04-01|     member|ff96777d-cfda-11e...|FEMALE|   3267|      16|  5100|30-35|  REGULAR|    E_Store|     3|  FROZEN FOOD|
|      2018-04-09|     member|00d89024-cfdb-11e...|FEMALE|   6748|       1|  1100|55-60|  REGULAR|    A_Store|     1|          PET|
|      2018-04-01|     member|00858b3b-cfdb-11e...|FEMALE|    420|       4|  5300|55-60|  REGULAR|    A_Store|     1|          PET|
|      2018-04-13|     member|001035ed-cfdb-11e...|FEMALE|   3178|      16|  5600|45-50|  REGULAR|    A_Store|     1|      GADGETS|
|      2018-04-15|     member|ff5fafeb-cfda-11e...|FEMALE|   2727|      19| 

In [4]:
df_2018.createOrReplaceTempView("df_2018_view")

In [5]:
spark.sql('''
  SELECT
    *
  FROM
    df_2018_view
  LIMIT
    5
  ''').show()

+----------------+-----------+--------------------+------+-------+--------+------+-----+---------+-----------+------+-------------+
|transaction_date|member_type|           member_id|gender|product|quantity|amount|  age|card_type|branch_name|region|category_name|
+----------------+-----------+--------------------+------+-------+--------+------+-----+---------+-----------+------+-------------+
|      2018-04-01|     member|ff96777d-cfda-11e...|FEMALE|   3267|      16|  5100|30-35|  REGULAR|    E_Store|     3|  FROZEN FOOD|
|      2018-04-09|     member|00d89024-cfdb-11e...|FEMALE|   6748|       1|  1100|55-60|  REGULAR|    A_Store|     1|          PET|
|      2018-04-01|     member|00858b3b-cfdb-11e...|FEMALE|    420|       4|  5300|55-60|  REGULAR|    A_Store|     1|          PET|
|      2018-04-13|     member|001035ed-cfdb-11e...|FEMALE|   3178|      16|  5600|45-50|  REGULAR|    A_Store|     1|      GADGETS|
|      2018-04-15|     member|ff5fafeb-cfda-11e...|FEMALE|   2727|      19| 

In [6]:
spark.sql('''
  SELECT
    COUNT(DISTINCT member_id)
  FROM
    df_2018_view
  ''').show()

+-------------------------+
|count(DISTINCT member_id)|
+-------------------------+
|                   356982|
+-------------------------+



In [7]:
spark.sql('''
  SELECT
    COUNT (DISTINCT member_id)
  FROM
    df_2018_view
  WHERE
    transaction_date BETWEEN '2018-04-01' AND '2018-04-30'
''').show()

+-------------------------+
|count(DISTINCT member_id)|
+-------------------------+
|                   101637|
+-------------------------+



In [8]:
# Sliced Data: JAN, FEB, MAR
spark.sql('''
WITH
customers_in_april AS (
SELECT
  DISTINCT member_id
FROM
  df_2018_view
WHERE
  transaction_date BETWEEN '2018-04-01' AND '2018-04-30'
)
,customers_in_april_joined_with_past_3_months_txn AS (
SELECT
  a.member_id
  ,b.quantity
  ,b.transaction_date
FROM
  customers_in_april AS a
LEFT JOIN
  df_2018_view AS b
ON
  a.member_id = b.member_id
WHERE
  b.transaction_date BETWEEN '2018-01-01' AND '2018-03-31'
)
,aggregated_total_txns_per_month AS (
SELECT
   member_id
  -- Jan transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-01-31' THEN quantity ELSE 0 END) AS pm3_total_txn
  -- Feb transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-02-01' AND '2018-02-28' THEN quantity ELSE 0 END) AS pm2_total_txn
  -- Mar transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-03-31' THEN quantity ELSE 0 END) AS pm1_total_txn
FROM
  customers_in_april_joined_with_past_3_months_txn
GROUP BY
  member_id
)
SELECT
  *
FROM
  aggregated_total_txns_per_month
LIMIT
  5
''').show()

+--------------------+-------------+-------------+-------------+
|           member_id|pm3_total_txn|pm2_total_txn|pm1_total_txn|
+--------------------+-------------+-------------+-------------+
|0070f220-cfdb-11e...|            8|            0|           33|
|00e2f0bd-cfdb-11e...|            0|            0|           13|
|ff222fee-cfda-11e...|            0|            0|           29|
|ffd94d9e-cfda-11e...|            0|            0|           67|
|ff864ac0-cfda-11e...|            0|            0|           11|
+--------------------+-------------+-------------+-------------+



In [9]:
# Sliced data: FEB, MAR, APR
spark.sql('''
WITH
customers_in_may AS (
SELECT
  DISTINCT member_id
FROM
  df_2018_view
WHERE
  transaction_date BETWEEN '2018-05-01' AND '2018-05-31'
)
,customers_in_may_joined_with_past_3_months_txn AS (
SELECT
  a.member_id
  ,b.quantity
  ,b.transaction_date
FROM
  customers_in_may AS a
LEFT JOIN
  df_2018_view AS b
ON
  a.member_id = b.member_id
WHERE
  b.transaction_date BETWEEN '2018-02-01' AND '2018-05-31'
)
,aggregated_total_txns_per_month AS (
SELECT
   member_id
  -- Feb transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-02-01' AND '2018-02-28' THEN quantity ELSE 0 END) AS pm4_total_txn
  -- Mar transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-03-31' THEN quantity ELSE 0 END) AS pm3_total_txn
  -- Apr transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-04-01' AND '2018-04-30' THEN quantity ELSE 0 END) AS pm2_total_txn
FROM
  customers_in_may_joined_with_past_3_months_txn
GROUP BY
  member_id
)
SELECT
  *
FROM
  aggregated_total_txns_per_month
LIMIT
  5
''').show()

+--------------------+-------------+-------------+-------------+
|           member_id|pm4_total_txn|pm3_total_txn|pm2_total_txn|
+--------------------+-------------+-------------+-------------+
|fefb42a6-cfda-11e...|           26|            0|            0|
|01283abc-cfdb-11e...|            9|            5|            0|
|010f5665-cfdb-11e...|           11|            0|            0|
|ff96c597-cfda-11e...|            0|            0|            0|
|00285114-cfdb-11e...|            0|            0|           13|
+--------------------+-------------+-------------+-------------+



In [10]:
# Slice Data: MAR, APR, MAY
spark.sql('''
WITH
customers_in_jun AS (
SELECT
  DISTINCT member_id
FROM
  df_2018_view
WHERE
  transaction_date BETWEEN '2018-06-01' AND '2018-06-30'
)
,customers_in_jun_joined_with_past_3_months_txn AS (
SELECT
  a.member_id
  ,b.quantity
  ,b.transaction_date
FROM
  customers_in_jun AS a
LEFT JOIN
  df_2018_view AS b
ON
  a.member_id = b.member_id
WHERE
  b.transaction_date BETWEEN '2018-03-01' AND '2018-06-30'
)
,aggregated_total_txns_per_month AS (
SELECT
   member_id
  -- Mar transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-02-31' THEN quantity ELSE 0 END) AS pm5_total_txn
  -- Apr transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-04-01' AND '2018-04-30' THEN quantity ELSE 0 END) AS pm4_total_txn
  -- May transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-05-01' AND '2018-05-31' THEN quantity ELSE 0 END) AS pm3_total_txn
FROM
  customers_in_jun_joined_with_past_3_months_txn
GROUP BY
  member_id
)
SELECT
  *
FROM
  aggregated_total_txns_per_month
LIMIT
  5
''').show()

+--------------------+-------------+-------------+-------------+
|           member_id|pm5_total_txn|pm4_total_txn|pm3_total_txn|
+--------------------+-------------+-------------+-------------+
|ff406689-cfda-11e...|            0|            0|           41|
|ff38edfc-cfda-11e...|            0|            0|            0|
|ffa89fdc-cfda-11e...|            0|            0|            0|
|fef351b2-cfda-11e...|            0|            0|            0|
|ff8203f1-cfda-11e...|            0|            0|            0|
+--------------------+-------------+-------------+-------------+



In [11]:
result = spark.sql('''
  WITH customers_per_month AS (
    SELECT DISTINCT member_id, '2018-04' AS PARTITION_MONTH
    FROM df_2018_view WHERE transaction_date BETWEEN '2018-04-01' AND '2018-04-30'

    UNION ALL

    SELECT DISTINCT member_id, '2018-05' AS PARTITION_MONTH
    FROM df_2018_view WHERE transaction_date BETWEEN '2018-05-01' AND '2018-05-31'

    UNION ALL

    SELECT DISTINCT member_id, '2018-06' AS PARTITION_MONTH
    FROM df_2018_view WHERE transaction_date BETWEEN '2018-06-01' AND '2018-06-30'
  ),
  transactions_history AS (
    SELECT
      c.member_id,
      c.PARTITION_MONTH,
      t.quantity,
      t.transaction_date
    FROM customers_per_month c
    LEFT JOIN df_2018_view t
    ON c.member_id = t.member_id
    WHERE t.transaction_date BETWEEN '2018-01-01' AND '2018-06-30'
  ),
  aggregated_txns AS (
    SELECT
      member_id,
      SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-01-31' THEN quantity ELSE 0 END) AS pm5_total_txn,
      SUM(CASE WHEN transaction_date BETWEEN '2018-02-01' AND '2018-02-28' THEN quantity ELSE 0 END) AS pm4_total_txn,
      SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-03-31' THEN quantity ELSE 0 END) AS pm3_total_txn,
      SUM(CASE WHEN transaction_date BETWEEN '2018-04-01' AND '2018-04-30' THEN quantity ELSE 0 END) AS pm2_total_txn,
      SUM(CASE WHEN transaction_date BETWEEN '2018-05-01' AND '2018-05-31' THEN quantity ELSE 0 END) AS pm1_total_txn,
      PARTITION_MONTH
    FROM transactions_history
    GROUP BY member_id, PARTITION_MONTH
  )
  SELECT * FROM aggregated_txns
''')

result.show(10)

+--------------------+-------------+-------------+-------------+-------------+-------------+---------------+
|           member_id|pm5_total_txn|pm4_total_txn|pm3_total_txn|pm2_total_txn|pm1_total_txn|PARTITION_MONTH|
+--------------------+-------------+-------------+-------------+-------------+-------------+---------------+
|ffdf410a-cfda-11e...|            0|            0|            0|           73|            0|        2018-04|
|009a2599-cfdb-11e...|            0|            0|            0|           68|            0|        2018-04|
|ffff6c82-cfda-11e...|            0|            0|            0|           30|            0|        2018-04|
|ffd15e3f-cfda-11e...|            0|           18|            0|           73|           35|        2018-04|
|ffd86309-cfda-11e...|            0|            0|            0|           19|            0|        2018-04|
|fffe81f0-cfda-11e...|            0|            0|            0|           21|            0|        2018-04|
|ff4659d3-cfda-11e.

In [12]:
spark.sql('''
WITH
customers_in_april AS (
SELECT
  DISTINCT member_id
FROM
  df_2018_view
WHERE
  transaction_date BETWEEN '2018-04-01' AND '2018-04-30'
)
,customers_in_april_joined_with_past_3_months_txn AS (
SELECT
  a.member_id
  ,b.quantity
  ,b.transaction_date
FROM
  customers_in_april AS a
LEFT JOIN
  df_2018_view AS b
ON
  a.member_id = b.member_id
WHERE
  b.transaction_date BETWEEN '2018-01-01' AND '2018-03-31'
)
,aggregated_total_txns_per_month_jan_mar AS (
SELECT
   member_id
  -- Jan transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-01-31' THEN quantity ELSE 0 END) AS pm3_total_txn
  -- Feb transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-02-01' AND '2018-02-28' THEN quantity ELSE 0 END) AS pm2_total_txn
  -- Mar transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-03-31' THEN quantity ELSE 0 END) AS pm1_total_txn
  ,'2018-04-01' AS PARTITION_MONTH
FROM
  customers_in_april_joined_with_past_3_months_txn
GROUP BY
  member_id
)

, customers_in_may AS (
SELECT
  DISTINCT member_id
FROM
  df_2018_view
WHERE
  transaction_date BETWEEN '2018-05-01' AND '2018-05-31'
)
,customers_in_may_joined_with_past_3_months_txn AS (
SELECT
  a.member_id
  ,b.quantity
  ,b.transaction_date
FROM
  customers_in_may AS a
LEFT JOIN
  df_2018_view AS b
ON
  a.member_id = b.member_id
WHERE
  b.transaction_date BETWEEN '2018-02-01' AND '2018-04-30'
)
,aggregated_total_txns_per_month_feb_apr AS (
SELECT
   member_id
  -- Feb transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-02-01' AND '2018-02-28' THEN quantity ELSE 0 END) AS pm3_total_txn
  -- Mar transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-03-31' THEN quantity ELSE 0 END) AS pm2_total_txn
  -- Apr transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-04-01' AND '2018-04-30' THEN quantity ELSE 0 END) AS pm1_total_txn
  ,'2018-05-01' AS PARTITION_MONTH
FROM
  customers_in_may_joined_with_past_3_months_txn
GROUP BY
  member_id
)


, customers_in_jun AS (
SELECT
  DISTINCT member_id
FROM
  df_2018_view
WHERE
  transaction_date BETWEEN '2018-06-01' AND '2018-06-30'
)
,customers_in_jun_joined_with_past_3_months_txn AS (
SELECT
  a.member_id
  ,b.quantity
  ,b.transaction_date
FROM
  customers_in_jun AS a
LEFT JOIN
  df_2018_view AS b
ON
  a.member_id = b.member_id
WHERE
  b.transaction_date BETWEEN '2018-03-01' AND '2018-05-31'
)
,aggregated_total_txns_per_month_mar_may AS (
SELECT
   member_id
  -- Mar transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-03-01' AND '2018-03-31' THEN quantity ELSE 0 END) AS pm3_total_txn
  -- Apr transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-04-01' AND '2018-04-30' THEN quantity ELSE 0 END) AS pm2_total_txn
  -- May transaction count
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-05-01' AND '2018-05-31' THEN quantity ELSE 0 END) AS pm1_total_txn
  ,'2018-06-01' AS PARTITION_MONTH
FROM
  customers_in_jun_joined_with_past_3_months_txn
GROUP BY
  member_id
)

, union_all_tables AS (
  SELECT * FROM aggregated_total_txns_per_month_jan_mar
  UNION
  SELECT * FROM aggregated_total_txns_per_month_feb_apr
  UNION
  SELECT * FROM aggregated_total_txns_per_month_mar_may
  )
SELECT
    PARTITION_MONTH,
    COUNT(member_id) AS total_member_id
FROM
    union_all_tables
GROUP BY
    PARTITION_MONTH
ORDER BY
    PARTITION_MONTH
''').show()

+---------------+---------------+
|PARTITION_MONTH|total_member_id|
+---------------+---------------+
|     2018-04-01|          59143|
|     2018-05-01|          84325|
|     2018-06-01|          89658|
+---------------+---------------+



In [13]:
spark.sql('''
WITH
customers_in_july AS (
SELECT
  DISTINCT member_id
FROM
  df_2018_view
WHERE
  transaction_date BETWEEN '2018-07-01' AND '2018-07-31'
)
,customers_in_july_joined_with_past_transactions AS (
SELECT
  a.member_id
  ,b.quantity
  ,b.amount
  ,b.transaction_date
FROM
  customers_in_july AS a
LEFT JOIN
  df_2018_view AS b
ON
  a.member_id = b.member_id
AND
  b.transaction_date BETWEEN '2018-01-01' AND '2018-06-30'
)
,aggregated_total_txns_per_month AS (
SELECT
   member_id
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-04-01' AND '2018-04-30' THEN quantity ELSE 0 END) AS pm3_total_txn
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-05-01' AND '2018-05-31' THEN quantity ELSE 0 END) AS pm2_total_txn
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-06-01' AND '2018-06-30' THEN quantity ELSE 0 END) AS pm1_total_txn
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-07-01' AND '2018-07-31' THEN quantity ELSE 0 END) AS pm0_total_txn
  ,'2018-07-01' AS PARTITION_MONTH
FROM
  customers_in_july_joined_with_past_transactions
GROUP BY
  member_id
)
,customers_with_target_definition AS (
SELECT
  *
  ,CASE
    WHEN pm1_total_txn = 0 AND pm2_total_txn = 0 AND pm3_total_txn = 0 THEN 1
    ELSE 0
    END AS target
FROM
  aggregated_total_txns_per_month
)
,customers_joined_with_features AS (
SELECT
  a.member_id
  ,b.quantity
  ,b.amount
  ,b.category_name
  ,b.transaction_date
FROM
  customers_in_july AS a
LEFT JOIN
  df_2018_view AS b
ON
  a.member_id = b.member_id
AND
  b.transaction_date BETWEEN '2018-01-01' AND '2018-03-31'
)
,aggregated_categories AS (
SELECT
   member_id

  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('ACCESSORIES') THEN amount ELSE 0 END) AS pm_accessories
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('APPLIANCES') THEN amount ELSE 0 END) AS pm_appliances
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('CLEANERS') THEN amount ELSE 0 END) AS pm_cleaners

  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('CLOTHES') THEN amount ELSE 0 END) AS pm_clothes
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('FOOD') THEN amount ELSE 0 END) AS pm_food
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('FROZEN FOOD') THEN amount ELSE 0 END) AS pm_frozen_food

  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('GADGETS') THEN amount ELSE 0 END) AS pm_gadgets
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('HARDWARE') THEN amount ELSE 0 END) AS pm_hardware
  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('PET') THEN amount ELSE 0 END) AS pm_pet

  ,SUM(CASE WHEN transaction_date BETWEEN '2018-01-01' AND '2018-03-31' AND category_name IN ('SHOES') THEN amount ELSE 0 END) AS pm_shoes

FROM
  customers_joined_with_features
GROUP BY
  member_id
)
,customers_with_target_joined_with_features AS (
SELECT
  a.member_id
  ,a.target
  ,b.pm_accessories
  ,b.pm_appliances
  ,b.pm_cleaners
  ,b.pm_clothes
  ,b.pm_food
  ,b.pm_frozen_food
  ,b.pm_gadgets
  ,b.pm_hardware
  ,b.pm_pet
  ,b.pm_shoes
FROM
  customers_with_target_definition AS a
LEFT JOIN
  aggregated_categories AS b
ON
  a.member_id = b.member_id
)

,count_customers_with_target_definition AS (
SELECT
  target
  ,COUNT(member_id) AS cnt_member_id
  ,COUNT(DISTINCT member_id) AS cntd_member_id
FROM
  customers_with_target_definition
GROUP BY
  target
)

SELECT
  *
FROM
  count_customers_with_target_definition
''').show()

+------+-------------+--------------+
|target|cnt_member_id|cntd_member_id|
+------+-------------+--------------+
|     1|        15873|         15873|
|     0|        34794|         34794|
+------+-------------+--------------+



In [14]:
34794/((15873+34794))

0.6867191663212742

In [15]:
# Churn rate in %
34794/((15873+34794))*100

68.67191663212742